In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
import sys
sys.path.append('/home/jovyan/Tjaernoe2022-group5/notebooks/Remy/')
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
start_year = 2035
end_year = 2050#can be the same as start for 1 year
exp_id = 'ssp585'

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

Historical:

In [4]:
#cat = col.search(source_id=['NorESM2-LM'], experiment_id=['historical'], table_id=['day'], variable_id=['hus','va'], member_id=['r1i1p1f1'])

Scenario:

In [5]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=[exp_id], table_id=['day'], variable_id=['hus','va'], member_id=['r1i1p1f1'])

In [7]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['historical'], table_id=['day'], variable_id=['clt'], member_id=['r1i1p1f1'])

In [8]:
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCC,NorESM2-LM,historical,r1i1p1f1,day,clt,gn,gs://cmip6/CMIP6/CMIP/NCC/NorESM2-LM/historica...,NaN,20190815


In [6]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [7]:
dataset_list = list(dset_dict.keys())

In [8]:
dset = dset_dict[dataset_list[0]]
dset = dset.sel(member_id='r1i1p1f1',time=slice(str(start_year)+"-01-01", str(end_year)+"-01-01"))

In [9]:
vas = dset.va
hus = dset.hus
plev = dset.plev
lat_ = hus.lat
lon_ = hus.lon

In [10]:
ivt = compute_ivt(hus,vas,plev)
dset.close()

In [11]:
ivt_ns = ivt.copy()
ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward

In [12]:
# pre-computed 93th percentile IVT
q93 = xr.open_dataset('q93_00-15.nc')
q93 = q93.rename({'__xarray_dataarray_variable__':'ivt'})

In [13]:
excess = ivt_ns_pos-q93

q93.close()
ivt_ns_pos.close()

ar_points = xr.where(excess>0,1,0)

In [14]:
out_ar = ar_points.copy()
ar_points.close()

out_ar = out_ar.drop_vars(['quantile','member_id'])

out_loc = np.zeros((out_ar.ivt.shape[0],out_ar.ivt.shape[1],out_ar.ivt.shape[2])).astype(int)

res_lon = abs(lon_.values[1]-lon_.values[0])
res_lat = abs(np.min(np.diff(lat_.values)))

test_val = out_ar.ivt.values[:]

for tt in range(len(out_ar.time)):
    df_loc = test_val[:,tt,:]
    ll = plt.contour(df_loc,levels=[0,1])
    plt.close()
    for item in ll.collections:
        for i in item.get_paths():
            v = i.vertices
            crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
            if (crit>=20): # AR has to be at least 20 deg lat 
                xx=(v[:, 0]).astype(int)
                yy=(v[:, 1]).astype(int)
                for (x,y) in zip(xx,yy):
                    out_loc[y,tt,x] = 1
    

/tmp/ipykernel_1878/1875809623.py:15: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])


In [15]:
out_ar.ivt.values = out_loc.astype(bool)

In [16]:
out_ar.to_netcdf(str(start_year)+str(end_year)+'_crit.nc')
out_ar.close()

In [17]:
AR = xr.open_dataset(str(start_year)+str(end_year)+'_crit.nc')

In [18]:
ivt_ = AR.ivt
ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon')# add extra points for AR at the edge of the world

In [19]:
out_ar = ivt.copy()
out_ar.values[:] = out_ar.values[:]*0.0

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        # for each latitude get limit longitudes of shapes
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1: # if there is a shape at that longitude
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10: # avoid filling where only 1 point in shape
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc


for k in range(ivt.shape[0]):
    matrix = ivt[k,:,:]
    tst = floodfill(matrix)
    out_ar[k,:,:] = tst
    
# add the values of the extra points to get ARs at the end of the world
out_ar[:,:,:5] = out_ar[:,:,:5]+out_ar[:,:,144:]
out_ar = out_ar[:,:,:144]
out_ar = xr.where(out_ar>1,1,out_ar)
out_ar = out_ar.astype(bool)

lat_ = AR.lat
lon_ = AR.lon

out_ar.to_netcdf(f'{exp_id}_{start_year}{end_year}_AR_detection.nc')